In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [3]:
print(y.shape, tX.shape, ids.shape)

(250000,) (250000, 30) (250000,)


In [4]:
from implementations import *

max_iters = 100
# least square
# gamma = 0.0000001
# logistic
gamma = 0.000001 
batch_size = 32
lambda_ = 0.1
# initial_w = np.random.rand(tX.shape[1]) * 2 - 1
initial_w = np.zeros(tX.shape[1])
# weights, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
# weights, loss = least_squares_SGD(y, tX, initial_w, batch_size, max_iters, gamma)
# weights, loss = least_squares(y, tX)
# weights, loss = ridge_regression(y, tX, lambda_)
# weights, loss = logistic_regression_GD(y, tX, initial_w, max_iters, gamma)
weights, loss = reg_logistic_regression_SGD(y, tX, lambda_, initial_w, batch_size, max_iters, gamma)

LS GD(0/99): loss=0.6368770942468809
LS GD(1/99): loss=0.6331223020396461
LS GD(2/99): loss=0.6361252867570109
LS GD(3/99): loss=0.6286009619300704
LS GD(4/99): loss=0.6364519931030792
LS GD(5/99): loss=0.6304636526031069
LS GD(6/99): loss=0.6221688059558363
LS GD(7/99): loss=0.6198192737484138
LS GD(8/99): loss=0.6213455391144651
LS GD(9/99): loss=0.6165356068488825
LS GD(10/99): loss=0.6314137876053305
LS GD(11/99): loss=0.6128198437379186
LS GD(12/99): loss=0.6267748276877136
LS GD(13/99): loss=0.6309778288274651
LS GD(14/99): loss=0.6641761874625774
LS GD(15/99): loss=0.6131373634932724
LS GD(16/99): loss=0.6206802233166234
LS GD(17/99): loss=0.6292849047693188
LS GD(18/99): loss=0.6461079671490194
LS GD(19/99): loss=0.6212038910608095
LS GD(20/99): loss=0.6091764629407846
LS GD(21/99): loss=0.6242090981010232
LS GD(22/99): loss=0.6074370493794209
LS GD(23/99): loss=0.6069610715286476
LS GD(24/99): loss=0.6135335211014981
LS GD(25/99): loss=0.611317620716925
LS GD(26/99): loss=0.60

In [5]:
y_train_pred = predict_labels(weights, tX)
print(np.sum(y_train_pred==y)/y_train_pred.shape[0])

0.659012


## Generate predictions and save ouput in csv format for submission:

In [6]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [7]:
OUTPUT_PATH = 'predictions.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)